In [1]:
import numpy as np
import pandas as pd
import tqdm

import catboost as cat
from catboost import CatBoostClassifier

## Загрузим данные

In [47]:
%%time
edges = pd.read_csv('./edges.csv')
ids = pd.read_csv('./ids.csv')
vertices = pd.read_csv('./vertices.csv')
okved = pd.read_csv('./okved.txt')
okved = okved.set_index('code')

CPU times: user 3.15 s, sys: 341 ms, total: 3.49 s
Wall time: 2.85 s


In [5]:
vertices['main_okved'] = vertices['main_okved'].astype(str)

In [12]:
df = edges.set_index('id_1').join(vertices.set_index('id'))
df = df.reset_index()
df= df.rename(columns={'index':'id_1', 'main_okved': 'main_okved_1', 'region_code':'region_code_1', 'company_type': 'company_type_1'})
df = df.set_index('id_2').join(vertices.set_index('id'))
df = df.reset_index()
df= df.rename(columns={'index':'id_2', 'main_okved': 'main_okved_2', 'region_code':'region_code_2', 'company_type': 'company_type_2'})
df

,id_2,id_1,value,n_transactions,main_okved_1,region_code_1,company_type_1,main_okved_2,region_code_2,company_type_2
0,492,176,362479.129002,1.000000,46.51,54,Limited,46.51,54,Limited
1,681,510,322006.537938,1.000000,46.0,78,Limited,16.23,78,Limited
2,853,565,385983.709663,80.997950,16.23,77,Limited,46.9,77,Limited
3,880,861,351371.294011,80.997950,47.91,66,Individual,32.3,66,Individual
4,898,278,320095.836906,1.000000,43.22,53,Limited,46.13,78,Limited
...,...,...,...,...,...,...,...,...,...,...
4685639,1534748,1440742,391652.249750,1.000000,46.9,77,Limited,74.2,33,Individual
4685640,1534748,1474388,320095.836906,1.000000,73.11,77,Individual,74.2,33,Individual
4685641,1534749,206964,435925.606915,186.749487,66.3,66,Actionary,68.2,66,Individual
4685642,1534749,692041,403521.505955,297.027591,43.21,66,Limited,68.2,66,Individual


In [13]:
from collections import Counter

c = Counter(df['id_1'].values)
df = df.set_index('id_1')


In [54]:
def most_popular_by_criteria(column_name):
    company_codes_all = []
    for p in c.most_common(100):
        table_company = (df.loc[p[0]]).sort_values(by=column_name, ascending=False).head(25)
        targets = table_company['main_okved_2']
        targets = targets.values
        company_codes_simple = [company.split('.')[0] for company in targets]
        company_set = set(company_codes_simple)
        #print(p[0], company_set, table_company['value'].sum(), df.loc[p[0]]['value'].sum())
        company_codes_all.extend(list(company_set))
    company_codes_all = Counter(company_codes_all)
    for p in company_codes_all.most_common(20):
        print("Код", p[0], "Частота", p[1], okved.loc[int(p[0])]['name'])
most_popular_by_criteria('n_transactions') #по числу транзакций

Код 46 Частота 82  Торговля оптовая кроме оптовой торговли автотранспортными средствами и мотоциклами
Код 47 Частота 73  Торговля розничная кроме торговли автотранспортными средствами и мотоциклами
Код 52 Частота 56  Складское хозяйство и вспомогательная транспортная деятельность
Код 62 Частота 52  Разработка компьютерного программного обеспечения консультационные услуги в данной области и другие сопутствующие услуги
Код 49 Частота 39  Деятельность сухопутного и трубопроводного транспорта
Код 45 Частота 39  Торговля оптовая и розничная автотранспортными средствами и мотоциклами и их ремонт
Код 68 Частота 36  Операции с недвижимым имуществом
Код 73 Частота 35  Деятельность рекламная и исследование конъюнктуры рынка
Код 69 Частота 34  Деятельность в области права и бухгалтерского учета
Код 43 Частота 31  Работы строительные специализированные
Код 63 Частота 27  Деятельность в области информационных технологий
Код 95 Частота 25  Ремонт компьютеров предметов личного потребления и хозяйстве

In [55]:
most_popular_by_criteria('value') #по денежкам

Код 46 Частота 88  Торговля оптовая кроме оптовой торговли автотранспортными средствами и мотоциклами
Код 47 Частота 67  Торговля розничная кроме торговли автотранспортными средствами и мотоциклами
Код 62 Частота 54  Разработка компьютерного программного обеспечения консультационные услуги в данной области и другие сопутствующие услуги
Код 49 Частота 42  Деятельность сухопутного и трубопроводного транспорта
Код 68 Частота 38  Операции с недвижимым имуществом
Код 52 Частота 38  Складское хозяйство и вспомогательная транспортная деятельность
Код 41 Частота 37  Строительство зданий
Код 73 Частота 36  Деятельность рекламная и исследование конъюнктуры рынка
Код 45 Частота 34  Торговля оптовая и розничная автотранспортными средствами и мотоциклами и их ремонт
Код 63 Частота 33  Деятельность в области информационных технологий
Код 43 Частота 31  Работы строительные специализированные
Код 70 Частота 29  Деятельность головных офисов; консультирование по вопросам управления
Код 69 Частота 27  Де

In [36]:
okved.head(20)

,code,name
0,1,Растениеводство и животноводство охота и пред...
1,2,Лесоводство и лесозаготовки
2,3,Рыболовство и рыбоводство
3,5,Добыча угля
4,6,Добыча сырой нефти и природного газа
5,7,Добыча металлических руд
6,8,Добыча прочих полезных ископаемых
7,9,Предоставление услуг в области добычи полезны...
8,10,Производство пищевых продуктов
9,11,Производство напитков


In [4]:
np.random.seed(7777)

## Обучим модель

In [5]:
result = pd.DataFrame(columns=['id_1', 'id_2'])

In [6]:
# для каждой вершины из ids с помощью catboost найдем 1000 самых вероятных ребер
for i in tqdm.tqdm(ids.id):
    # соберем датасет из всех возможных вершин
    # вершины имеющие в исходных данных ребро с i обозначим 1, остальные 0
    # учтем то, что вершина i может быть как среди id_1, так и среди id_2
    df1 = edges[edges['id_1'] == i].reset_index()
    df2 = edges[edges['id_2'] == i].reset_index()

    df = df1[['id_2', 'id_1']].rename(columns={'id_1':'id_2', 'id_2':'id_1'}).append(df2[['id_1', 'id_2']])
    df['target'] = 1
    
    df = vertices.set_index('id').join(df.set_index('id_1')['target']).fillna(0)
    
    
    X = df[['main_okved', 'region_code', 'company_type']]
    y = df['target']
    
    model = CatBoostClassifier(iterations=100, verbose=False)
    cat_features = [0,1,2] # все признаки категориальные
    
    model.fit(X, y, cat_features)

    preds = model.predict_proba(X)[:,1]

    df['preds'] = preds
    df['id_2'] = i
    
    # возьмем первую 1000 предсказанных ребер, исключив те, про которые мы уже знали
    res = df[df['target'] != 1].sort_values(by='preds', ascending=False).iloc[:1000].reset_index()[['id', 'id_2']]
    res.columns = ['id_1', 'id_2']
    
    result = result.append(res, ignore_index=True, sort=False)

 28%|██▊       | 28/100 [26:00<1:05:35, 54.66s/it]

Training has stopped (degenerate solution on iteration 93, probably too small l2-regularization, try to increase it)


100%|██████████| 100/100 [1:32:34<00:00, 54.28s/it]


## Результат готов к отправке

In [7]:
result.to_csv('submission.csv', index = False)